In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
from fl_g13.config import RAW_DATA_DIR


from torchvision import datasets, transforms

from fl_g13 import dataset as dataset_handler

import torch
import torch.nn as nn
import torch.nn.functional as F


import flwr
from flwr.common import Context

from flwr.simulation import run_simulation


2025-04-20 12:02:47.163 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Load data

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])
cifar100_train = datasets.CIFAR100(root=RAW_DATA_DIR, train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root=RAW_DATA_DIR, train=False, download=True, transform=transform)

In [6]:
### train val split
train_dataset,val_dataset = dataset_handler.train_test_split(cifar100_train,train_ratio=0.8)

In [7]:
# I.I.D Sharding Split
## k client
k =10
clients_dataset_train= dataset_handler.iid_sharding(train_dataset,k)
clients_dataset_val= dataset_handler.iid_sharding(val_dataset,k)

## Tiny model

In [8]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))     # -> [B, 16, 32, 32]
        x = F.max_pool2d(x, 2)        # -> [B, 16, 16, 16]
        x = F.relu(self.conv2(x))     # -> [B, 32, 16, 16]
        x = F.max_pool2d(x, 2)        # -> [B, 32, 8, 8]
        x = x.view(x.size(0), -1)     # -> [B, 32*8*8]
        x = self.fc1(x)               # -> [B, 100]
        return x

## Init model , optimizer and loss function

In [9]:
net = TinyCNN().to(DEVICE)
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4, weight_decay=0.04)
criterion = torch.nn.CrossEntropyLoss()

# Define the ClientApp

## Build module local

Build module local such that ClientApp can use it

In [50]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4649 sha256=a588eeccbccdbea56dbfe92e55bd612babd81e07366f39e361ccc99a5ffdec08
  Stored in directory: C:\Users\ADMIN\AppData\Local\Temp\pip-ephem-wheel-cache-dbhxfe7i\wheels\b7

## create FlowerClient instances  

In [40]:
'''
Function load data client is to simulate the distribution data into each client
In the real case, each client will have its dataset
'''
def load_data_client(context: Context):
    partition_id = context.node_config["partition-id"] 
    print(f"Client {partition_id} is ready to train")
    trainloader = DataLoader(clients_dataset_train[partition_id])
    valloader = DataLoader(clients_dataset_val[partition_id])
    return trainloader, valloader

### Create instant of ClientApp

In [41]:
from fl_g13.fl_pytorch.client_app import get_client_app

config={'local-epochs':1}
client = get_client_app(load_data_client,model=net,optimizer=optimizer,criterion=criterion,device=DEVICE,config=config)

# Define the Flower ServerApp

Customize built-in strategy Federated Averaging (FedAvg) of Flower to combine hyperparams in server-side and save model for each k epoch

The strategy could also incremental training an

## Create instant of ServerApp

In [42]:

from pathlib import Path
from torch.utils.data import DataLoader
from fl_g13.fl_pytorch.server_app import get_server_app

def get_datatest_fn(context: Context):
    return DataLoader(cifar100_test)

## checkpoints directory
current_path = Path.cwd()
model_test_path = current_path / "../models/model_test"
model_test_path.resolve()


num_rounds=2
save_every =1
fraction_fit=1.0  # Sample 100% of available clients for training
fraction_evaluate=0.5  # Sample 50% of available clients for evaluation
min_fit_clients=10  # Never sample less than 10 clients for training
min_evaluate_clients=5  # Never sample less than 5 clients for evaluation
min_available_clients=10  # Wait until all 10 clients are available
device=DEVICE
use_wandb=False


server = get_server_app(checkpoint_dir=model_test_path.resolve(),
                        model_class=TinyCNN,
                        optimizer=optimizer,criterion=criterion, get_datatest_fn=get_datatest_fn,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit, 
                        fraction_evaluate=fraction_evaluate,  
                        min_fit_clients=min_fit_clients,  
                        min_evaluate_clients=min_evaluate_clients, 
                        min_available_clients=min_available_clients, 
                        device=device,
                        use_wandb=use_wandb,
                        save_every=save_every
                        )

⚠️ No checkpoint found at C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\model_test. Creating a new model.


# Run the training


In [43]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"]= {"num_cpus": 1, "num_gpus": 0.25}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [44]:
NUM_CLIENTS =10

In [45]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]


Continue train model from epoch 1


INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      ROUND 0💡 New best global model found: 0.009500
INFO :      initial parameters (loss, other metrics): 4.609353698635101, {'centralized_accuracy': 0.0095}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


Saving best centralized model at epoch 0...
💾 Saved checkpoint at: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\model_test\TinyCNN\FL_TinyCNN_best.pth


(ClientAppActor pid=26496) 2025-04-20 14:22:00.855 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


...

INFO :      ROUND 2💡 New best global model found: 0.130200
INFO :      fit progress: (2, 3.9105881756424905, {'centralized_accuracy': 0.1302}, 129.56632340000215)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Saving best centralized model at epoch 2...
💾 Saved checkpoint at: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\model_test\TinyCNN\FL_TinyCNN_best.pth
(ClientAppActor pid=26284) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=26284) 	📊 Training Loss: 4.0742
(ClientAppActor pid=26284) 	✅ Training Accuracy: 9.20%
(ClientAppActor pid=26284) 	⏳ Elapsed Time: 8.71s | ETA: 0.00s
(ClientAppActor pid=26284) 	🕒 Completed At: 14:23
(ClientAppActor pid=26496) Client 6 is ready to train
(ClientAppActor pid=26284) Client 0 is ready to train


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 134.12s
INFO :      	History (loss, distributed):
INFO :      		round 1: 4.20430006866455
INFO :      		round 2: 3.9495217295527456
INFO :      	History (loss, centralized):
INFO :      		round 0: 4.609353698635101
INFO :      		round 1: 4.174223951470852
INFO :      		round 2: 3.9105881756424905
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.0768), (2, 0.115)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': [(0, 0.0095), (1, 0.0848), (2, 0.1302)]}
INFO :      


(ClientAppActor pid=26284) Client 9 is ready to train [repeated 3x across cluster]


(ClientAppActor pid=20452) 2025-04-20 14:22:00.944 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13 [repeated 3x across cluster]


In [51]:
server = get_server_app(checkpoint_dir=model_test_path.resolve(),
                        model_class=TinyCNN,
                        optimizer=optimizer,criterion=criterion, get_datatest_fn=get_datatest_fn,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit, 
                        fraction_evaluate=fraction_evaluate,  
                        min_fit_clients=min_fit_clients,  
                        min_evaluate_clients=min_evaluate_clients, 
                        min_available_clients=min_available_clients, 
                        device=device,
                        use_wandb=use_wandb,
                        save_every=save_every
                        )

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\model_test\FL_TinyCNN_epoch_2.pth
📦 Model class in checkpoint: TinyCNN
✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\model_test\FL_TinyCNN_epoch_2.pth, resuming at epoch 3


In [52]:
# Run simulation second time
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Continue train model from epoch 3


INFO :      ROUND 0💡 New best global model found: 0.177600
INFO :      initial parameters (loss, other metrics): 3.6569207283109426, {'centralized_accuracy': 0.1776}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=19884) 2025-04-20 14:38:29.197 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


(ClientAppActor pid=19884) Client 3 is ready to train
(ClientAppActor pid=2940) No prefix/name for the model was provided, choosen prefix/name: cheeky_pidgey_11
(ClientAppActor pid=25656) 
(ClientAppActor pid=19884) 
(ClientAppActor pid=9928) 
(ClientAppActor pid=2940) 
(ClientAppActor pid=19884) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=19884) 	📊 Training Loss: 3.6465
(ClientAppActor pid=19884) 	✅ Training Accuracy: 16.70%
(ClientAppActor pid=19884) 	⏳ Elapsed Time: 13.74s | ETA: 0.00s
(ClientAppActor pid=19884) 	🕒 Completed At: 14:38
(ClientAppActor pid=19884) 
(ClientAppActor pid=25656) Client 1 is ready to train [repeated 3x across cluster]
(ClientAppActor pid=9928) No prefix/name for the model was provided, choosen prefix/name: sleepy_squirtle_38 [repeated 3x across cluster]
(ClientAppActor pid=9928) 
(ClientAppActor pid=2940) 
(ClientAppActor pid=25656) 
(ClientAppActor pid=19884) 
(ClientAppActor pid=25656) 
(ClientAppActor pid=9928) 
(ClientAppActor pid=2940) 
(Client

INFO :      aggregate_fit: received 10 results and 0 failures


...

INFO :      ROUND 2💡 New best global model found: 0.203500
INFO :      fit progress: (2, 3.4974401091873646, {'centralized_accuracy': 0.2035}, 118.90462670000124)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=19884) Client 5 is ready to train
(ClientAppActor pid=9928) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=9928) 	📊 Training Loss: 3.5808
(ClientAppActor pid=9928) 	✅ Training Accuracy: 17.93%
(ClientAppActor pid=9928) 	⏳ Elapsed Time: 8.69s | ETA: 0.00s
(ClientAppActor pid=9928) 	🕒 Completed At: 14:40


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 124.46s
INFO :      	History (loss, distributed):
INFO :      		round 1: 3.651327788695693
INFO :      		round 2: 3.562506917529553
INFO :      	History (loss, centralized):
INFO :      		round 0: 3.6569207283109426
INFO :      		round 1: 3.567416850231588
INFO :      		round 2: 3.4974401091873646
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.1666), (2, 0.1804)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': [(0, 0.1776), (1, 0.192), (2, 0.2035)]}
INFO :      


(ClientAppActor pid=9928) Client 8 is ready to train [repeated 4x across cluster]


(ClientAppActor pid=25656) 2025-04-20 14:38:29.191 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13 [repeated 3x across cluster]
